In [45]:
import pandas as pd

In [46]:
with open('tweets.csv', 'r') as f:
    df = pd.read_csv(f)

In [47]:
df = df.dropna()

In [48]:
# Задание 1

In [49]:
df[df['author_name'].apply(lambda x: len(x.split())) > 1] \
                    .groupby('author_name') \
                    .agg({'tweet_content': lambda x: min(x, key=len)})

,tweet_content
author_name,
'(Robert Smith),"A lecture on the spirit of the computer, recen..."
(((E. Glen Weyl))),Couldn't be more excited to put out probably m...
(((Matthew Lewis))),"""This could be huge.""\n\nFolks, this *is* huge..."
(◔_◔) ¯\_(ツ)_/¯,@EdLatimore The problem is twofold:\n\n1. Suga...
1jack 🌱,Institutional money is going to move into this...
...,...
🔥 #Author Abraham Lopez,"@EdLatimore Misery doesn’t just love company, ..."
🔭🌃Jana Grcevich 📡🚀,Hey #ancientegypt twitter: I would love to kno...
🔱Master of Ecstasy🔱,I HAVE A NEW FAVORITE WORD:\n\nAPOTHEOSIS:\n\n...


In [50]:
# Задание 2

In [51]:
import emoji

In [52]:
emoji_list = df['tweet_content'].apply(emoji.emoji_lis).sum()

In [53]:
emoji_df = pd.DataFrame(emoji_list)
emoji_df['emoji'].value_counts().head(30)

✅    117
😂    101
👇    101
🔥     52
🙏     41
🏻     41
🏽     38
👏     34
❤     32
♂     30
🚨     30
🇸     30
🇺     29
🏼     26
👨     20
⚫     19
👍     18
👩     18
💯     17
🤔     17
🏾     17
🆕     16
🙌     16
💪     15
♀     15
🚀     15
👉     14
🙄     13
👦     13
⤵     13
Name: emoji, dtype: int64

In [54]:
# Задание 3

In [55]:
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [56]:
with open('positive-words.txt', 'r', encoding='latin-1') as pos, \
     open('negative-words.txt', 'r', encoding='latin-1') as neg:
    positive = set(pos.read().splitlines()[35:])
    negative = set(neg.read().splitlines()[35:])

lemmatizer = WordNetLemmatizer()

In [86]:
def count_sentiment(tweet, positive=positive, negative=negative, lemmatizer=lemmatizer):
    
    word_list = nltk.word_tokenize(tweet)
    score = 0
    c_pos = Counter()
    c_neg = Counter()
    
    for word in word_list:
        word = lemmatizer.lemmatize(word)
        if word in positive:
            score += 1
            c_pos.update([word])
        elif word in negative:
            score -= 1
            c_neg.update([word])
    
    return score, c_pos, c_neg

In [91]:
counted = df['tweet_content'].apply(count_sentiment)
df['sentiment_score'] = counted.apply(lambda x: x[0])
df['positive_words'] = counted.apply(lambda x: x[1])
df['negative_words'] = counted.apply(lambda x: x[2])
df.head()

,author_name,created_at,handle,likes,retweets,tweet_content,sentiment_score,positive_words,negative_words
0,Naval,2019-08-07 22:36:56,naval,7566,1498,"Unresolved thoughts, prematurely pushed out of...",-2,{},"{'chronic': 1, 'anxiety': 1}"
1,Naval,2019-08-07 05:00:38,naval,21886,5984,The modern mind is overstimulated and the mode...,2,{'modern': 2},{}
2,Naval,2019-08-07 04:52:33,naval,6462,1266,The Lindy Effect for startups:\n\nThe longer y...,0,{},{}
3,Naval,2019-08-06 08:35:26,naval,466,61,@orangebook_ This was a good tweet.,1,{'good': 1},{}
4,Naval,2019-08-06 07:33:20,naval,3971,906,Social media lowers the cost of raising &amp; ...,-1,{'win': 1},"{'extremism': 1, 'outraged': 1}"


In [78]:
top = df[df['sentiment_score'] > 5][['tweet_content', 'sentiment_score']]
top.sort_values(by='sentiment_score', ascending=False)

,tweet_content,sentiment_score
10518,Truth lies in a bit of this and a bit of that....,12
519,Doctors won’t make you healthy.\n\nNutritionis...,10
29478,"Men, DO YOUR DUTY each day...\n\n- maximum eff...",10
24374,Being kind to others is the most effective way...,9
2539,"""If you have one good move that gains a clear ...",9
...,...,...
12959,"Whether is physical or mental performances, me...",6
12825,Financial freedom is not a goal.\nFinancial fr...,6
12780,You can’t love anyone as long as you haven’t b...,6
12732,A simple indicator to ascertain that I’m not i...,6


In [79]:
bottom = df[df['sentiment_score'] < -5][['tweet_content', 'sentiment_score']]
bottom.sort_values(by='sentiment_score', ascending=True)

,tweet_content,sentiment_score
18682,Being a founder is hard. Being an early employ...,-12
3626,When a naive investor who saw no risk suddenly...,-11
3365,When a naive investor who saw no risk suddenly...,-11
3243,When a naive investor who saw no risk suddenly...,-11
24316,"We unconsciously flee from our problems, seeki...",-10
...,...,...
13074,"As long as you are afraid of being alone,\n\na...",-6
12712,A common mistake of parents is to hide unprais...,-6
12697,Youth is the greatest asset you have in your 2...,-6
14925,"""The vision of the left, full of envy and rese...",-6


In [93]:
df[df['sentiment_score'] > 0]['positive_words'].sum().most_common(10)

[('like', 1373),
 ('good', 968),
 ('work', 944),
 ('better', 623),
 ('right', 618),
 ('best', 603),
 ('great', 530),
 ('love', 499),
 ('enough', 370),
 ('free', 341)]

In [94]:
df[df['sentiment_score'] < 0]['negative_words'].sum().most_common(10)

[('problem', 555),
 ('fear', 317),
 ('hard', 313),
 ('bad', 299),
 ('wrong', 254),
 ('risk', 188),
 ('lose', 183),
 ('mistake', 177),
 ('hate', 171),
 ('lie', 168)]